In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

## Summery
1. Reading the files
2. Considerations
3. FE
# 

#### 1. Reading the files

In [6]:
df = pd.read_csv('../data/data_eda.csv',parse_dates={'order_date':['date']})

In [9]:
df.dtypes

order_date               datetime64[ns]
customer_id                      object
order_hour                        int64
customer_order_rank             float64
is_failed                         int64
voucher_amount                  float64
delivery_fee                    float64
amount_paid                     float64
restaurant_id                     int64
city_id                           int64
payment_id                        int64
platform_id                       int64
transmission_id                   int64
is_returning_customer             int64
is_l2015                           bool
is_voucher_ge30                   int64
is_voucher_ge20_le30              int64
is_delivery_fee_ge5               int64
dtype: object

In [10]:
df.set_index('customer_id', inplace=True)

#### 2.Considerations
Since this is a transaction dataset and we need to predict the churn for each user, I think some aggregated features with some features from the last transaction could be helpful. Bad experiences could lead to churn. The aggregation functions that I'm considering are `count`, `mean`, `min`, `max`, `std` of different features(not for all of them). Also, some time features like for how long the user is inactive could also be helpful. Also, I extracted some date features from the first `order_date` of the customer. These features could give us some information about some potential campaign that leads some customers to order, and the quality of those people in long run. Also We `sum` the times that a customer used our extracted features from EDA.

#### 3.FE

In [18]:
max_day = df.order_date.max()

In [11]:
gp = df.groupby(df.index)

In [21]:
%%time

agg = gp.agg(
    cnt = ('order_date', 'count'),
    year = ('order_date', lambda x: x.iloc[0].year),
    month = ('order_date', lambda x: x.iloc[0].month),
    day = ('order_date', lambda x: x.iloc[0].day),
    duration = ('order_date', lambda x: (x.max() - x.min()).days),
    days_since_last_order = ('order_date', lambda x: (max_day - x.max()).days),

    failures = ('is_failed', 'sum'),
    last_order_failed = ('is_failed', lambda x: x.iloc[-1]),
    
    voucher_amount_n = ('voucher_amount', lambda x: (x!=0).sum()),
    
    voucher_amount_mean = ('voucher_amount', 'mean'),
    voucher_amount_max = ('voucher_amount', 'max'),
    voucher_amount_min = ('voucher_amount', 'min'),
    voucher_amount_std = ('voucher_amount', 'std'),
    last_order_with_voucher = ('voucher_amount',lambda x: np.where(x.iloc[-1] > 0, 1, 0)),
    
    delivery_fee_mean = ('delivery_fee', 'mean'),
    delivery_fee_std = ('delivery_fee', 'std'),
    delivery_fee_min = ('delivery_fee', 'min'),
    delivery_fee_max = ('delivery_fee', 'max'),
    last_order_delivery_fee = ('delivery_fee', lambda x: x.iloc[-1]),
    
    amount_paid_mean = ('amount_paid', 'mean'),
    amount_paid_min = ('amount_paid', 'min'),
    amount_paid_max = ('amount_paid', 'max'),
    amount_paid_std = ('amount_paid', 'std'),
    last_order_amount_paid = ('amount_paid', lambda x: x.iloc[-1]),
    
    restaurant_id_n = ('restaurant_id', 'nunique'),
    last_restaurant = ('restaurant_id', lambda x : x.iloc[-1]),
    
    city_id_n = ('city_id', 'nunique'),
    last_city = ('city_id', lambda x : x.iloc[-1]),

    payment_id_n = ('payment_id','nunique'),
    last_order_payment_id = ( 'payment_id', lambda x: x.iloc[-1]),
    platform_id_n = ('platform_id' , 'nunique'),
    last_order_platform_id = ('platform_id', lambda x: x.iloc[-1]),
    transmission_id_n = ('transmission_id', 'nunique'),
    last_order_transmission_id = ('transmission_id', lambda x: x.iloc[-1]),
    
    is_l2015 = ('is_l2015', 'sum'),
    is_voucher_ge30 = ('is_voucher_ge30', 'sum'),           
    is_voucher_ge20_le30 = ('is_voucher_ge20_le30', 'sum'),
    is_delivery_fee_ge5=('is_voucher_ge20_le30', 'sum'),
    
)

CPU times: user 4min 34s, sys: 1.42 s, total: 4min 35s
Wall time: 4min 35s


In [48]:
agg.head()

,cnt,year,month,day,duration,days_since_last_order,failures,last_order_failed,voucher_amount_n,voucher_amount_mean,...,payment_id_n,last_order_payment_id,platform_id_n,last_order_platform_id,transmission_id_n,last_order_transmission_id,is_l2015,is_voucher_ge30,is_voucher_ge20_le30,is_delivery_fee_ge5
customer_id,,,,,,,,,,,,,,,,,,,,,
000097eabfd9,1,2015,6,20,0,618,0,0,0.0,0.0,...,1,1779,1,30231,1,4356,0,0,0,0
0000e2c6d9be,1,2016,1,29,0,395,0,0,0.0,0.0,...,1,1619,1,30359,1,4356,0,0,0,0
000133bb597f,1,2017,2,26,0,1,0,0,0.0,0.0,...,1,1619,1,30359,1,4324,0,0,0,0
00018269939b,1,2017,2,5,0,22,0,0,0.0,0.0,...,1,1619,1,30359,1,4356,0,0,0,0
0001a00468a6,1,2015,8,4,0,573,0,0,0.0,0.0,...,1,1619,1,29463,1,4356,0,0,0,0


In [42]:
#for for categorical features with not lots of unique values we can calculate number of occurrence for each 
def unstack_gp(df, col): 
    tmp = gp[col].value_counts().unstack()
    tmp.columns = [col+'_'+str(c) for c in tmp.columns]
    return tmp.reset_index(level=-1).fillna(0)


In [43]:
#for for categorical features with not lots of unique values we can calculate number of occurrence for each 
payment_id_cnt = unstack_gp(gp, 'payment_id')
payment_id_cnt.head()

,customer_id,payment_id_1491,payment_id_1523,payment_id_1619,payment_id_1779,payment_id_1811
0,000097eabfd9,0.0,0.0,0.0,1.0,0.0
1,0000e2c6d9be,0.0,0.0,1.0,0.0,0.0
2,000133bb597f,0.0,0.0,1.0,0.0,0.0
3,00018269939b,0.0,0.0,1.0,0.0,0.0
4,0001a00468a6,0.0,0.0,1.0,0.0,0.0


In [44]:
platform_id_cnt = unstack_gp(gp, 'platform_id')
platform_id_cnt

,customer_id,platform_id_525,platform_id_22167,platform_id_22263,platform_id_22295,platform_id_29463,platform_id_29495,platform_id_29751,platform_id_29815,platform_id_30135,platform_id_30199,platform_id_30231,platform_id_30359,platform_id_30391,platform_id_30423
0,000097eabfd9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0000e2c6d9be,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,000133bb597f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,00018269939b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0001a00468a6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245450,fffd696eaedd,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245451,fffe9d5a8d41,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245452,ffff347c3cfa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
245453,ffff4519b52d,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
transmission_id_cnt =  unstack_gp(gp, 'transmission_id')
transmission_id_cnt

,customer_id,transmission_id_212,transmission_id_1988,transmission_id_2020,transmission_id_4196,transmission_id_4228,transmission_id_4260,transmission_id_4324,transmission_id_4356,transmission_id_4996,transmission_id_21124
0,000097eabfd9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0000e2c6d9be,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,000133bb597f,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,00018269939b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0001a00468a6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
245450,fffd696eaedd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
245451,fffe9d5a8d41,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
245452,ffff347c3cfa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
245453,ffff4519b52d,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [47]:
#for for categorical features with not lots of unique values we can calculate number of occurrence for each 
payment_id_cnt = gp.payment_id.value_counts().unstack().reset_index(level=-1).fillna(0)
payment_id_cnt.head()

payment_id,customer_id,1491,1523,1619,1779,1811
0,000097eabfd9,0.0,0.0,0.0,1.0,0.0
1,0000e2c6d9be,0.0,0.0,1.0,0.0,0.0
2,000133bb597f,0.0,0.0,1.0,0.0,0.0
3,00018269939b,0.0,0.0,1.0,0.0,0.0
4,0001a00468a6,0.0,0.0,1.0,0.0,0.0


In [ ]:
#to see how much a customer trying new resturants
agg['new_restaurant_ratio'] = agg.restaurant_id_n / agg['cnt']